## Linked open data

This notebook experiments with querying linked open data in a Jupyter Notebook rendered through Quarto. 

The Python code below queries the NFDI4Culture Wikibase at https://wikibase.wbworkshop.tibwiki.io/wiki/Main_Page and returns data based on the ID inputted by the user. 

This executes in Jupyter Notebook which is able to run the Python code and provides a static output when saved in the Notebook. It cannot be executed dynamically in Quarto since the Quarto front-end does not support stdin input requests.

To reset the output, run 'Kernel > Restart kernel and clear all outputs'.

In [ ]:
import requests

# Global variables
endpoint_url = 'https://wikibase.wbworkshop.tibwiki.io'
resource_url = '/w/api.php'

entity = input("What's the Q number (including the 'Q')? ")
print('Check out ' + endpoint_url + '/wiki/' + entity + ' to see the GUI.')
resourceUrl = '/w/api.php?action=wbgetclaims&format=json&entity='+entity
uri = endpoint_url + resourceUrl
r = requests.get(uri)
data = r.json()
claims = data['claims']
print('subject: ', entity)
print()
for property, values in claims.items():
    print('property: ', property)
    for value in values:
        try:
            # print Q ID if the value is an item
            print('item value: ', value['mainsnak']['datavalue']['value']['id'])
        except:
            try:
                # print the string value if the value is a literal
                print('literal value: ', value['mainsnak']['datavalue']['value'])
            except:
                # print the whole snak if the value is something else
                print('other value: ', value['mainsnak'])
    print()

The below Python code experiments with using SPARQLWrapper to retrieve data based on a SPARQL query.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wbworkshop.tibwiki.io")

sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?item ?itemLabel ?url
WHERE { 
    ?item tibt:P1 tib:Q15.
    ?item tibt:P22 ?url
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
} 
""")

# Convert results to JSON format
sparql.setReturnFormat(JSON)
result = sparql.query().convert()
